<p align="center">
<img src="http://wandb.me/logo-im-png" width="800" alt="Weights & Biases" />
</p>

# 🪐 Operation REBOOT: Mission Start

Welcome, **Neural Architect**. The ship's AI core is down. Your job: fine-tune a foundational model with astrological Q&A data to restore its deep space reasoning abilities.

**Your mission:**
- Adjust dataset splits
- Configure training arguments
- Launch training and monitor with **Weights & Biases (W&B)**
- Test and evaluate your fine-tuned model

All systems go. Let's bring this vessel back online.

#### Install and Import  

In [ ]:
!pip install transformers datasets accelerate wandb trl huggingface_hub -q
!pip install bitsandbytes peft -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import wandb

import json
import math
import random
from pathlib import Path


import torch
import torchvision
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    EarlyStoppingCallback
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel,
)
from huggingface_hub import login
from google.colab import userdata

## 🔌 Connect Neural Telemetry (W&B Setup)

In [ ]:
# When prompted to authorize your
# wandb: You can find your API key in your browser here: https://wandb.ai/authorize
# wandb: Paste an API key from your profile and hit enter:

#Login to W&B - If not using Colab to store/fetch key comment next two lines and just run wandb.login() command
wandb_key = userdata.get('WANDB_API_KEY')
wandb.login(key=wandb_key)
#wandb.login()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammadbakir (m-bakir) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
WANDB_PROJECT_NAME = "Astros-FT-Workshop"
WANDB_ENTITY = "FT-Testing" #Set your W&B Entity #TODO - Change to None for final release

## 🧪 Dataset Control Room
Adjust the dataset splits and prepare the astrological QA dataset for training.

## 🌎 Initialize Experiment, Read Data, Split Data ☄️

In this section, we:

* Retrieve the Astros Dataset from [W&B Registry FC_FT_Workshop_Dataset collection](https://wandb.ai/orgs/FullyConnected-2025-Workshops/registry/dataset?selectionPath=fullyconnected-2025-workshops%2Fwandb-registry-dataset%2FFC_FT_Workshop_Dataset&view=versions) TODO: Update for final version
* Load the Astros Dataset containing universe-related Q&A data.
* Create prompts from the question/answer pairs & load into a pandas dataframe
* Convert the pandas DataFrame into a Hugging Face Dataset.

✅ All the heavy lifting is done here automatically — no manual setup needed

#### Helper functions to load and prepare the dataset

In [ ]:
#Load Animations
run = wandb.init(entity=WANDB_ENTITY,
                  project=WANDB_PROJECT_NAME,
                  job_type="animation_retrieval",
                  name=f"fetch_animations",
                  settings=wandb.Settings(silent=True)
                  )

# Download Animations
animate_artifact = run.use_artifact('wandb-registry-FT-Workshop-Collatoral/Animation-Scripts:v1', type='code')
animate_dir = animate_artifact.download()

run.finish()

#### Helper functions to load and prepare the dataset

In [ ]:
# Load and validate the JSONL dataset
def load_jsonl_dataset(file_path):
    """
    Load a JSONL dataset into a pandas DataFrame.

    Each line is parsed as a JSON object.
    Handles and reports JSON decoding errors without stopping execution.

    Args:
        file_path (str): Path to the JSONL file.

    Returns:
        pd.DataFrame: DataFrame containing the loaded dataset.
    """
    data = []
    error_count = 0
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f, start=1):
            line = line.strip()
            if line:
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"[Error] Line {i}: Could not decode JSON.")
                    print(f"Content: {line}")
                    print(f"Error: {str(e)}\n")
                    error_count += 1
                    continue
    if error_count > 0:
        print(f"\n⚠️ Finished loading with {error_count} decoding errors. Please check your dataset formatting!")
    else:
        print("✅ Successfully loaded dataset with no errors.")
    return pd.DataFrame(data)

# Create a prompt format from each question-answer pair
def create_prompt(row):
    return f"Question: {row['question']}\nAnswer: {row['answer']}"

# Utility function to reload and prepare the dataset when needed
def reload_astros_dataset(artifact_dir, filename="astro_dataset_train.jsonl"):
    """
    Reload the Astros dataset from a downloaded artifact directory.

    Args:
        artifact_dir (str or Path): Path to the artifact directory.
        filename (str): Name of the JSONL dataset file. Defaults to "astro_dataset_train.jsonl".

    Returns:
        tuple: A pandas DataFrame and a Hugging Face Dataset ready for training.
    """
    dataset_path = Path(artifact_dir) / filename
    df = load_jsonl_dataset(str(dataset_path))
    df['text'] = df.apply(create_prompt, axis=1)
    return df, Dataset.from_pandas(df)

def load_and_prepare_dataset(artifact_dir, filename, dataset_type="dataset"):
    """
    Load a JSONL dataset, create prompts, and convert to Hugging Face Dataset format.

    Args:
        artifact_dir (str or Path): Path to the artifact directory
        filename (str): Name of the JSONL file to load
        dataset_type (str): Type of dataset (e.g., "training" or "evaluation")

    Returns:
        tuple: (pandas DataFrame, Hugging Face Dataset)
    """
    print(f"\nLoading {dataset_type} dataset...")
    df = load_jsonl_dataset(str(Path(artifact_dir) / filename))
    df['text'] = df.apply(create_prompt, axis=1)
    hf_dataset = Dataset.from_pandas(df)
    print(f"✅ {dataset_type.capitalize()} dataset loaded with {len(df)} examples")
    return df, hf_dataset

#### Let's prepare our training dataset

In [ ]:
# Step 1: Initialize W&B run and download dataset
print("Step 1: Downloading dataset from Weights & Biases...")
run = wandb.init(entity=WANDB_ENTITY,
                 project=WANDB_PROJECT_NAME,
                 job_type="data_retrieval",
                 name="fetch_astros_dataset"
                 )

# Download the dataset artifact
artifact = run.use_artifact('wandb-registry-dataset/FC_FT_Workshop_Dataset:v4', type='dataset')
dataset_dir = artifact.download()
run.finish()
print("✅ Dataset downloaded successfully!")

# Step 2: Load and prepare datasets
df_train, training_dataset = load_and_prepare_dataset(dataset_dir, "astro_dataset_train.jsonl", "training") #look at the helper fuctions if you're interested in how we prepare the data

# Print dataset statistics
print("\nDataset Statistics:")
print(f"Training examples: {len(df_train)}")
print("\nExample prompt format:")
print(df_train['text'].iloc[0])

Step 1: Downloading dataset from Weights & Biases...


wandb:   1 of 1 files downloaded.  


✅ Dataset downloaded successfully!

Loading training dataset...
✅ Successfully loaded dataset with no errors.
✅ Training dataset loaded with 1600 examples

Dataset Statistics:
Training examples: 1600

Example prompt format:
Question: What are 'Superluminous Supernovae' (SLSNe) and what distinguishes Type I SLSNe from normal Type Ia supernovae spectroscopically?
Answer: Superluminous Supernovae (SLSNe) are much more luminous than normal Type Ia supernovae. Spectroscopically, Type I SLSNe are characterized by the absence of hydrogen and strong helium lines near peak light (like normal SNe Ia), but they show strong, broad metal lines, often including oxygen, magnesium, and calcium. Normal SNe Ia are defined by the presence of strong silicon absorption lines (Si II λ6355) near peak light, which are often weak or absent in SLSNe I. The differences in spectra indicate different progenitor systems and explosion mechanisms: SNe Ia are thermonuclear disruptions of white dwarfs, while SLSNe I ar

### 🌌 Dataset Loaded Successfully!

At this point, we've:
* Retrieved the Astros Dataset artifact
* Loaded it into a pandas DataFrame
* Created prompt-style text for fine-tuning
* Converted it into a Hugging Face Dataset for training

✨ Feel free to pause and explore the data before moving forward!

Exploring the dataset can help you:

* Understand the kinds of questions and answers the model will learn from
* Check for any strange patterns, formatting issues, or interesting insights
* Discover Easter Eggs

🛡️ We've added soft error handling while loading, so if you accidentally modify the dataset file, you'll be warned if any loading issues happen.

👉 Quick Tip: You don't need to modify the dataset to proceed, but if you want to explore, you can run things like:

```
print(df_train.sample(5))
print(df_train['question'].apply(len).describe())
print(df_train['answer'].apply(len).describe())
```

When you're ready, move on to loading the model and tokenizing the dataset!

## 🧠 Model Vault: Load & Configure the Neural Core

## 🚀 Load Pretrained Model and Prepare Dataset for Fine-Tuning 🌠
In this section, we:

* Retrieve the Model to Finetune from [W&B Registry FC_FT_Workshop_Model](https://wandb.ai/orgs/FullyConnected-2025-Workshops/registry/model?selectionPath=fullyconnected-2025-workshops%2Fwandb-registry-model%2FFC_FT_Workshop_Model&view=versions) TODO: Update for final version
* Select and load a pretrained language model and its tokenizer from Hugging Face.
* Format the Astros prompts into tokenized input IDs the model can understand.
* Apply padding and truncation to keep sequence lengths manageable.
* Split the tokenized dataset into training and validation sets (90% train / 10% validation).
* Define a compute_metrics function to track TODO during fine-tuning.

✅ All the setup for model loading, tokenization, and data splitting is handled for you — no manual steps required!

#### Helper functions to Load and configure a qunatized model

In [ ]:
def select_model():
    """
    Display a menu of available models and let the user select one.

    Returns:
        tuple: (model_name, model_version)
    """
    models = {
        "1": ("falcon-rw-1b", "v0"),
        "2": ("TinyLlama", "v2"),
    }

    print("\nAvailable Models:")
    print("1. Falcon RW 1B (v0)")
    print("2. TinyLlama (v2)")

    while True:
        choice = input("\nSelect a model (1-2): ").strip()
        if choice in models:
            model_name, version = models[choice]
            print(f"\n✅ Selected: {model_name} (version {version})")
            return model_name, version
        print("Invalid choice. Please select 1 or 2.")


def load_model_and_tokenizer():
    """
    Load the selected model and tokenizer from Weights & Biases with QLoRA configuration.

    Returns:
        tuple: (model, tokenizer, model_name)
    """
    # Let user select the model
    model_name, version = select_model()

    # Initialize W&B run
    print(f"\nStep 1: Downloading {model_name} model from Weights & Biases...")
    run = wandb.init(entity=WANDB_ENTITY,
                     project=WANDB_PROJECT_NAME,
                     job_type="model_retrieval",
                     name=f"fetch_{model_name}_model_for_finetuning"
                     )

    # Download model
    model_artifact = run.use_artifact(f'wandb-registry-model/FC_FT_Workshop_Model:{version}', type='model')
    model_dir = model_artifact.download()

    run.finish()
    print("✅ Model downloaded successfully!")

    # Load tokenizer
    print("\nStep 2: Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(Path(model_dir))
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Tokenizer loaded successfully!")

    # Configure 4-bit quantization
    print("\nStep 3: Configuring QLoRA...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    # Load model with quantization
    print("Loading model with 4-bit quantization...")
    model = AutoModelForCausalLM.from_pretrained(
        Path(model_dir),
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True    )


    # Prepare model for k-bit training
    model = prepare_model_for_kbit_training(model)

    # Configure LoRA
    lora_config = LoraConfig(
        r=16,  # rank
        lora_alpha=32,
        target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    # Get PEFT model
    model = get_peft_model(model, lora_config)

    # Print trainable parameters
    model.print_trainable_parameters()
    print("✅ Model loaded with QLoRA successfully!")
    #%run {animate_dir}/celebrate_model.py
    #celebrate_model(model.base_model.model.__class__.__name__)

    return model, tokenizer, model_name

wandb:   2 of 2 files downloaded.  


### Select Model
You will be prompted to select one of the following models

*   Option 1: [falcon-rw-1b](https://huggingface.co/tiiuae/falcon-rw-1b)
*   Option 2: [TinyLlama](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0)


In [ ]:
model, tokenizer, model_name = load_model_and_tokenizer()


Available Models:
1. Falcon RW 1B (v0)
2. TinyLlama (v2)

Select a model (1-2): 1

✅ Selected: falcon-rw-1b (version v0)

Step 1: Downloading falcon-rw-1b model from Weights & Biases...


wandb: Downloading large artifact FC_FT_Workshop_Model:v0, 2505.10MB. 12 files... 
wandb:   12 of 12 files downloaded.  
Done. 0:0:21.3 (117.5MB/s)


✅ Model downloaded successfully!

Step 2: Loading tokenizer...
✅ Tokenizer loaded successfully!

Step 3: Configuring QLoRA...
Loading model with 4-bit quantization...


You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


trainable params: 12,582,912 || all params: 1,324,208,128 || trainable%: 0.9502
✅ Model loaded with QLoRA successfully!


✅ Model and tokenizer successfully loaded from the artifact!

Next, we'll make a few adjustments to ensure the model handles padding correctly,
and then prepare our dataset for training by tokenizing the input prompts.


## 🔄 Tokenize & Split: Format Data for Finetuning

#### Helper fuctions for tokenizing datasets

In [ ]:
# Set pad token and pad token ID if missing (important for consistent model behavior)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# Function to tokenize input prompts for training
def tokenize_function(examples):
    """
    Tokenize the text prompts with padding and truncation.

    Args:
        examples (dict): A batch of examples with 'text' field.

    Returns:
        dict: Tokenized outputs (input_ids, attention_mask, etc.)
    """
    tokenized = tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Will be adding a few metric functions users can use to test out out the model - TODO: Cleanup , either remove or add addititions functions - not currently used
def compute_perpexity(eval_preds):
    """
    Compute perplexity metric from model evaluation logits.

    Args:
        eval_preds (tuple): Tuple of (logits, labels) from evaluation step.

    Returns:
        dict: Dictionary containing 'perplexity' score.
    """
    logits, labels = eval_preds
    if not isinstance(logits, torch.Tensor):
        logits = torch.tensor(logits)
    if not isinstance(labels, torch.Tensor):
        labels = torch.tensor(labels)

    # Shift logits and labels for causal language modeling
    shift_logits = logits[:, :-1, :].contiguous()
    shift_labels = labels[:, 1:].contiguous()

    # Compute cross-entropy loss
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(
        shift_logits.view(-1, shift_logits.size(-1)),
        shift_labels.view(-1)
    )
    perplexity = math.exp(loss.item()) if loss.item() < 100 else float("inf")
    return {"perplexity": perplexity}

def tokenized_train_test(training_dataset, split):
  # Apply the tokenizer to the datasets
    split_dataset = training_dataset.train_test_split(test_size=split)
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    train_dataset = train_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=train_dataset.column_names,
    )

    eval_dataset = eval_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=eval_dataset.column_names,
    )

    print("✅ Tokenization applied to Training & Evaluation Datasets successfully!")

    return train_dataset, eval_dataset


#### Load the datasets

You can modify how our training data is passed to our training script to finetune the model. Make sure to analyze the data so you can select an appropriate **Sample Size** and  **Train/Test split** for the finetuning process.

TO-DO: Update the values below for your first run. You can come back and try it with new values again.

In [ ]:
training_sample = training_dataset.shuffle(seed=42).select(range(1000)) # chosse between 100 and 1600 samples

train_test_split = 0.1 # choose a float value between 0 and 1

train_dataset, eval_dataset = tokenized_train_test(training_dataset, train_test_split)

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

✅ Tokenization applied to Training & Evaluation Datasets successfully!


## ⚙️ Training Command Center
Set training arguments to guide your model's learning trajectory.

## 🛰️ Training Arguments (Where You Fine-Tune Settings) 🌙

This is where you'll do most of your experimentation! 🎯

The `TrainingArguments` object controls how your model is fine-tuned, including:

* Batch size
* Number of epochs
* Learning rate
* Warmup steps
* Mixed precision (fp16) for faster training
* Checkpoint saving
* Reporting to Weights & Biases

You can modify the hyperparameters here to see how different settings impact model performance.

🔥 Pro Tip: TODO INSERT SOME TIPS


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    run_name=f"fine-tuning-{model.__class__.__name__}-qlora",
    output_dir="./results",
    num_train_epochs=10,
    #The following is commented out for now, used by me for optimization runs
    # per_device_train_batch_size=64,
    # per_device_eval_batch_size=32,
    # dataloader_num_workers=4,
    # gradient_accumulation_steps=16,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    dataloader_num_workers=1,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    do_train=True,
    do_eval=True,
    fp16=True,
    bf16=False,
    gradient_checkpointing=False, #Choose to store the full forward-pass activations in GPU RAM
    gradient_checkpointing_kwargs={"use_reentrant": False},
    group_by_length=True,
    report_to=["wandb"],
    remove_unused_columns=True,
    dataloader_pin_memory=True,
    optim="adamw_torch", #See https://huggingface.co/docs/transformers/v4.51.3/en/perf_train_gpu_one#optimizers
    learning_rate=2e-4,
    lr_scheduler_type="cosine", # See https://huggingface.co/docs/transformers/en/main_classes/optimizer_schedules#transformers.SchedulerType
    auto_find_batch_size=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=5,
    load_best_model_at_end=True,
    # metric_for_best_model="perplexity",
    metric_for_best_model="eval_loss",
    logging_strategy="steps",
    label_names=["labels"],
)


## 🛰️ Engage Training Tracker
Launch the model and track training live with W&B.

## 🔭 Initialize Trainer, Train, and Save 🌎

In this final section:

* We initialize the Trainer with:
  * The model
  * The datasets (train/test splits)
  * The training arguments
  * A data collator for language modeling
  * Our compute_metrics function to calculate TODO

* We start training by calling trainer.train().
* We save the fine-tuned model and tokenizer locally.
* We finish the W&B run to close the logging cleanly.

🧠 Reminder: After training finishes, your fine-tuned model will be available in your local runtime — you can upload it back to W&B or Hugging Face later!

🚨 Training Ahead: Be ready for 10-15 min runtimes with the default configs!


In [ ]:
#Configure model for training
model.config.use_cache = False  # Disable cache during training

# Set label names for PEFT model
model.config.label_names = ["labels"]

# Initialize trainer with modified configuration
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
        pad_to_multiple_of=8  # Add padding to multiple of 8 for better performance
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    #compute_metrics=compute_perpexity, #If monitoring additional metric(s) can set this argument accordingly to your compute functions
)

# Enable gradient checkpointing with the new format
if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()

#Working Idea - Launch training via animation - #TODO Figure out how to pipe the ignite button or just write simple message to run next cell to train model
def ignite_training():
    print("🚀 Fine-tuning started...")
    trainer.train()

%run {animate_dir}/launch_sequence.py
launch_sequence(training_args)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


## ⚙ Now we kick off the training process ⚙

In [ ]:
#Train
run = wandb.init(entity=WANDB_ENTITY,
                  project=WANDB_PROJECT_NAME,
                  job_type="finetuning_job",
                  name=f"fine_tune_{model.__class__.__name__}"
                  )
train_output = trainer.train()

wandb.finish()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,2.369300,2.229677
2,2.124200,2.068879
3,1.911000,1.991888
4,1.740700,1.975890
5,1.580000,1.974862
6,1.442900,2.003130
7,1.327000,2.037671
8,1.253300,2.059721


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

eval/loss,█▄▁▁▁▂▃▃
eval/runtime,█▂▂▁▁▂▇▂
eval/samples_per_second,▁▇▇██▇▂▇
eval/steps_per_second,▁▇▇██▇▂▇
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇███
train/grad_norm,▂▁▁▂▃▄▄▅▆▆▆▇█▇████
train/learning_rate,▃▇███▇▇▇▆▅▅▄▄▃▂▂▁▁
train/loss,█▇▅▅▄▄▄▄▄▃▃▂▂▂▂▂▁▁
eval/loss,2.05972
eval/runtime,4.745


## 💾 Save & Upload
Preserve your fine-tuned model as a W&B artifact.

We will now save this model to W&B. You will need the artifact refernce for this model to get points for completing the fine tuning section.

Tracking your model in W&B can be really helpful:

- You can now share this model with your team and beyond
- W&B creates a lineage map of your model so you can see the full model lifecycle dataset->training->final state

In [ ]:
#Saving and uploading best model - working draft
trainer.save_model("best_model")
tokenizer.save_pretrained("best_model")

from pathlib import Path
import wandb

run = wandb.init(project="Astros-FT-Workshop",
                 entity="FT-Testing",
                 job_type="FT-Workshop-Finetuning-Best-Model-Upload",
                 name="FT-Best-Model-Upload")

artifact = wandb.Artifact(
    name="ft-best-model",
    type="model",
    description="""Best model from the Astros-FT-Workshop."""
)

artifact.add_dir(local_path="./best_model")

logged_artifact = run.log_artifact(artifact)

run.link_artifact(
  artifact=logged_artifact,
  target_path="wandb-registry-model/FC_FT_Workshop_FineTuned_Models"
)

run.finish()

wandb: Adding directory to artifact (./best_model)... Done. 0.1s


## ✅ Mission Checkpoint: Model Finetuned

Congratulations, Architect! You've:
- Loaded and prepped your training dataset ✅
- Configured a foundational model ✅
- Finetuned it with parameter-efficient methods ✅
- Logged your training runs and saved the final model to Weights & Biases ✅

Your model is now part of your mission's neural infrastructure.

Next, we prepare to test and evaluate. But first, a quick system check...

## 🧰 Systems Maintenance Bay: Utilities

Before testing, it's wise to flush memory and check your hardware status. Use these utilities to prepare the environment.

Just like a good engineer, make sure the ship's neural bays are cleared and ready.

## Utilities 🧰

In [ ]:
# -- Flush out GPU memory - when required - may require restarting the notebook
import gc, torch

try:
    del trainer
except: print("cannot release memory")
try:
    del model
except: print("cannot release memory")
try:
    del tokenizer
except: print("cannot release memory")

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

wandb.finish()

In [ ]:
!nvidia-smi

Mon May 12 21:21:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             49W /  400W |    2051MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 🧪 Testing the Neural Core

Now that your model is trained and uploaded, it’s time to test your ship’s new neural core.

You’ll load the fine-tuned model and run test prompts to ensure it responds with precision and depth—critical for deep-space operations.

We’ve equipped you with a call function wrapped in `Weave`, our GenAI interface and telemetry layer.

# 🔧 Testing our model 🪛

Let's start by creating some helper functions to load and call the model we just trained.

Since we created an adapter during the finetuning process, our load model function loads the original model along with our adapter using PEFT.

## 🛰️ Introducing Weave: Your AI Telemetry and Evaluation Suite

**Weave** is Weights & Biases’ next-gen platform for tracking, evaluating, and visualizing GenAI applications.

In REBOOT, you'll use Weave to:
- Log and score model generations
- Run structured evaluations on Q&A performance
- Compare outputs with reference answers

This enables you to **quantitatively assess** how mission-ready your model is.

Let’s initialize Weave and plug it into your finetuned system.

In [ ]:
!pip install weave "weave[scorers]" -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
import weave
weave.init('e2e-test')

  if event.key is 'enter':



Logged in as Weights & Biases user: anushrav-vatsa.
View Weave data at https://wandb.ai/wandb-smle/e2e-test/weave


#### Helper functions to load our local model

In [ ]:
def load_finetuned_model(adapter_dir, base_model_dir):

    tokenizer = AutoTokenizer.from_pretrained(adapter_dir, use_fast=True)

    # Load base model

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    # Load model with quantization
    print("Loading model with 4-bit quantization...")
    base_model = AutoModelForCausalLM.from_pretrained(
        Path(base_model_dir),
        quantization_config=bnb_config,
        device_map="auto",
        )
    # Load fine-tuned adapter
    model = PeftModel.from_pretrained(base_model, adapter_dir)
    return tokenizer, model

### Calling our Local Finetuned Model

In [ ]:
@weave.op()
def call_model(question: str) -> str:
    """Generate an answer from your Local LLM given a prompt."""

    system_prompt = "You are an expert in astrophysics. Please provide a concise and truthful answer to the following question:"
    prompt = system_prompt + "\n\n" + question + "\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=150, do_sample=False, eos_token_id=tokenizer.eos_token_id, pad_token_id=model.config.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True).replace(prompt, '').strip()

## 📊 Final Check: Evaluation Protocols

Your neural core is active—but is it mission-grade?

Use this section to:
- Load an evaluation dataset
- Score model responses using embedding similarity
- Track performance with W&B + Weave

**Evaluation is critical** before deployment—it ensures your model’s reasoning is aligned with mission parameters.

# Evaluating our Model

## Load model

In [ ]:
base_model_dir = "./artifacts/falcon-rw-1b:v0" # add path
adapter_dir = "./best_model" #add path to adapter dir

tokenizer, model = load_finetuned_model(adapter_dir, base_model_dir)

Loading model with 4-bit quantization...


## Get Eval Dataset

In [ ]:
weave.init('fc25-wandb-admins/uncategorized')
eval_dataset_public_v0 = weave.ref('eval_dataset_public:v0').get()

Logged in as Weights & Biases user: anushrav-vatsa.
View Weave data at https://wandb.ai/fc25-wandb-admins/uncategorized/weave


## Test the model with a sample from our eval dataset

In [ ]:
question = eval_dataset_public_v0[20]['question']
answer = call_model(question)

print("🛰️  Incoming Transmission — Mission Q&A\n")
print(f"🧠 Question:\n{question}\n")
print(f"🤖 Model Response:\n{answer}")

🍩 https://wandb.ai/fc25-wandb-admins/uncategorized/r/call/0196c660-c6c9-7422-b585-d8b8214f818d
🛰️  Incoming Transmission — Mission Q&A

🧠 Question:
How can 'cosmic variance' affect the interpretation of measurements of the cosmic star formation rate density?

🤖 Model Response:
Cosmic variance refers to the fact that the observed number of stars in a given cosmic volume (e.g., the Milky Way or the universe as a whole) depends on the initial density and initial velocity distribution of stars. As stars form, they disperse, merge, and disperse again. This process introduces random fluctuations into the observed number of stars, which can lead to large uncertainties in the measured star formation rate density. The standard error of the cosmic star formation rate density (SFRD) is the integral of the integrated density over the redshift range where the observed number of stars is most likely to lie. The standard error depends on the initial density (Ω_0) and the initial velocity (Ω_0 ∝ Ω_


## Setup eval

In [ ]:
import asyncio
from weave.scorers import EmbeddingSimilarityScorer
similarity_scorer = EmbeddingSimilarityScorer(
    model_id="openai/text-embedding-3-small",  # will need to update this to bedrock's titan models
    threshold=0.7
)

similarity_scorer.column_map = {
    "output": "model_output",  # Your model's response
    "target": "answer", # The reference response (expected)
    "kwargs": "question"
}

In [ ]:
eval_dataset = eval_dataset_public_v0.rows[20:25] # select samples to run evalaution against

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')# will remove this once we move to bedrock

evaluation = weave.Evaluation(
   dataset=eval_dataset, scorers=[similarity_scorer], name="model_qna_eval"
   )

print(await(evaluation.evaluate(call_model)) )

Error getting code deps for <function Evaluation.summarize at 0x78b0d8ef47c0>: invalid syntax (<unknown>, line 389)


🍩 https://wandb.ai/fc25-wandb-admins/uncategorized/r/call/0196c660-e994-7f43-ae1a-152a2ae366b5


Evaluated 1 of 5 examples

Evaluated 2 of 5 examples

Evaluated 3 of 5 examples

Evaluated 4 of 5 examples

Evaluated 5 of 5 examples

Evaluation summary
{
    'EmbeddingSimilarityScorer': {
        'similarity_score': {'mean': 0.7869259271583562},
        'is_similar': {'true_count': 4, 'true_fraction': 0.8}
    },
    'model_latency': {'mean': 46.71491947174072}
}

{'EmbeddingSimilarityScorer': {'similarity_score': {'mean': 0.7869259271583562}, 'is_similar': {'true_count': 4, 'true_fraction': 0.8}}, 'model_latency': {'mean': 46.71491947174072}}
